In [ ]:
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
from fastai.utils.show_install import *
show_install()



```text
=== Software === 
python        : 3.6.9
fastai        : 1.0.60
fastprogress  : 0.2.2
torch         : 1.3.1
nvidia driver : 418.67
torch cuda    : 10.1.243 / is available
torch cudnn   : 7603 / is enabled

=== Hardware === 
nvidia gpus   : 1
torch devices : 1
  - gpu0      : 15079MB | Tesla T4

=== Environment === 
platform      : Linux-4.14.137+-x86_64-with-Ubuntu-18.04-bionic
distro        : #1 SMP Thu Aug 8 02:47:02 PDT 2019
conda env     : Unknown
python        : /usr/bin/python3
sys.path      : 
/env/python
/usr/lib/python36.zip
/usr/lib/python3.6
/usr/lib/python3.6/lib-dynload
/usr/local/lib/python3.6/dist-packages
/usr/lib/python3/dist-packages
/usr/local/lib/python3.6/dist-packages/IPython/extensions
/root/.ipython
```

Please make sure to include opening/closing ``` when you paste into forums/github to make the reports appear formatted as code sections.

Optional package(s) to enhance the diagnostics can be installed with:
pip install distro
Once installed, re-run thi

In [ ]:
import cv2 as cv
import numpy as np
import glob

In [ ]:
def optflow_gen(video_dir, normalize = True):
  # The video feed is read in as a VideoCapture object
  cap =  cv.VideoCapture(video_dir)

  data_name = (video_dir.split("/")[-1]).split(".")[0]

  number_of_frame = int(cap.get(cv.CAP_PROP_FRAME_COUNT))

  # ret = a boolean return value from getting the frame, first_frame = the first frame in the entire video sequence
  ret, first_frame = cap.read()

  # Converts frame to grayscale because we only need the luminance channel for detecting edges - less computationally expensive
  prev_gray = cv.cvtColor(first_frame, cv.COLOR_BGR2GRAY)

  # Creates an frame filled with zero intensities with the same dimensions as the frame
  data_x_axis = np.zeros((first_frame.shape[0], first_frame.shape[1], number_of_frame))
  data_y_axis = np.zeros((first_frame.shape[0], first_frame.shape[1], number_of_frame))
  frame_num = 0

  while(cap.isOpened()):
    frame_num += 1
    # ret = a boolean return value from getting the frame, frame = the current frame being projected in the video
    ret, frame = cap.read()

    # Converts each frame to grayscale. Except part will active when all frames of video feed into network and input of cv.cvtColor will be empty
    try:
      gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    except :
      break

    # Calculates dense optical flow by Farneback method
    # https://docs.opencv.org/3.0-beta/modules/video/doc/motion_analysis_and_object_tracking.html#calcopticalflowfarneback
    flow = cv.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    
    # Normalization
    if normalize == True:
        flow_min = flow.min(axis=(0, 1), keepdims=True)
        flow_max = flow.max(axis=(0, 1), keepdims=True)
        flow = (flow - flow_min)/(flow_max-flow_min)
    
    data_x_axis[:,:,frame_num] = flow[:,:,0]
    data_y_axis[:,:,frame_num] = flow[:,:,1]

    # Updates previous frame
    prev_gray = gray
    # Frames are read by intervals of 1 millisecond. The programs breaks out of the while loop when the user presses the 'q' key
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
        
  # The following frees up resources and closes all windows
  cap.release()
  cv.destroyAllWindows()
  # We wil remove first frame because it is all zero
  return data_x_axis[...,1:], data_y_axis[...,1:], data_name
  

In [ ]:
def dataset_generate(raw_data, num_frames_seq = 3, step_of_move_along_height = 16, data_width = 48):
  # raw_data: A numpy ndarray resulted from optflow_gen(video_dir)
  # num_frames_seq: It shows that how many frames you want to predict next frame
  # step_of_move_along_height: In generating dataset, the step of move along height of each frame
  # data_width: Width of each sample of dataset 

  dataset_X = []
  label_X = []

  for j in range(num_frames_seq,raw_data.shape[2]):
      for i in range(0, raw_data.shape[0]-data_width+1, step_of_move_along_height):
         b = range(i, i+data_width)
         X_input = np.zeros((data_width,raw_data.shape[1],num_frames_seq))
         X_label = np.zeros((data_width,raw_data.shape[1],1))
        
         for t in range(num_frames_seq):
           X_input [...,t] = raw_data[b,:,j+t-num_frames_seq]
         X_label = raw_data[b,...,j]
        
         dataset_X.append(X_input)
         label_X.append (X_label)

  dataset_X = np.asarray(dataset_X, dtype=np.float64)
  label_X = np.asarray(label_X, dtype=np.float64)
  
  # First frame that was all zeros removed in optflow_gen section
  #dataset_X = dataset_X[16:,...] 
  # prepration resize for convlstm
  dataset_X = np.transpose(np.expand_dims((dataset_X), axis =1), (0, 4, 2, 3, 1))

  #label_X = label_X[16:,...]
  label_X= np.expand_dims(label_X, axis =-1)

  return dataset_X, label_X
  

In [ ]:
def save_dataset(dataset, label, save_dir, data_name):
   # dataset_X: First ret of optflow_gen(video_dir)
   # label_X: Second ret of optflow_gen(video_dir)
   # save_dir: Direction for saving dataset
   # data_name: Third ret of optflow_gen(video_dir)
   
   dataset = np.split(dataset, len(dataset), axis=0)
   label = np.split(label, len(label), axis=0)
  
   for j in range(len(dataset)):
     temp_dataset = dataset[j]
     temp_label = label[j]

     np.save(save_dir + "/dataset_" + data_name + str(j), temp_dataset)
     np.save(save_dir + "/label_" + data_name + str(j), temp_label)

     print("created " + data_name + str(j))

   print("Creating dataset from " + str(data_name) + " video is done")


In [ ]:
def normalizer(data_dir):    
    # data_dir: Direction of  dataset
    
    flow = sorted(glob.glob(data_dir + "/dataset" + "*" + ".npy"))
    label = sorted(glob.glob(data_dir + "/label" + "*" + ".npy"))

    for i in flow:
      dataset_X = np.load(i)
      dataset_X_min = dataset_X.min(axis=(2, 3), keepdims=True)
      dataset_X_max = dataset_X.max(axis=(2, 3), keepdims=True)
      dataset_X_norm = (dataset_X - dataset_X_min)/(dataset_X_max-dataset_X_min)
      np.save(i, dataset_X_norm)
      del dataset_X, dataset_X_min, dataset_X_max, dataset_X_norm

    for j in label:
      label_X = np.load(j)
      label_X_min = label_X.min(axis=(1, 2), keepdims=True)
      label_X_max = label_X.max(axis=(1, 2), keepdims=True)
      label_X_norm = (label_X - label_X_min)/(label_X_max-label_X_min)
      np.save(j, label_X_norm)
      del label_X, label_X_min, label_X_max, label_X_norm
      

In [ ]:
video_dir = "/content/drive/My Drive/Videos/bus.y4m"
save_dir_X = "/content/drive/My Drive/temporary_X"
#save_dir_Y = "/content/drive/My Drive/temporary_Y"

In [ ]:
data_x, _, data_name = optflow_gen(video_dir)
#data_x, data_y, data_name = optflow_gen(video_dir)

In [ ]:
dataset_X, label_X = dataset_generate(data_x)
#dataset_Y, label_Y = dataset_generate(data_y)

In [ ]:
save_dataset(dataset_X, label_X, save_dir_X, data_name)
#save_dataset(dataset_Y, label_Y, save_dir_Y, data_name)
del data_x, data_name, dataset_X, label_X

In [ ]:
# normalizer(save_dir_X)
# normalizer(save_dir_Y)

In [ ]:
video_dir = "/content/drive/My Drive/Videos/akiyo.y4m"
data_x, _, data_name = optflow_gen(video_dir)
dataset_X, label_X = dataset_generate(data_x)
save_dataset(dataset_X, label_X, save_dir_X, data_name)
del data_x, data_name, dataset_X, label_X

In [ ]:
video_dir = "/content/drive/My Drive/Videos/foreman.y4m"
data_x, _, data_name = optflow_gen(video_dir)
dataset_X, label_X = dataset_generate(data_x)
save_dataset(dataset_X, label_X, save_dir_X, data_name)
del data_x, data_name, dataset_X, label_X

In [ ]:
video_dir = "/content/drive/My Drive/Videos/coastguard.y4m"
data_x, _, data_name = optflow_gen(video_dir)
dataset_X, label_X = dataset_generate(data_x)
save_dataset(dataset_X, label_X, save_dir_X, data_name)
del data_x, data_name, dataset_X, label_X

In [ ]:
video_dir = "/content/drive/My Drive/Videos/container.y4m"
data_x, _, data_name = optflow_gen(video_dir)
dataset_X, label_X = dataset_generate(data_x)
save_dataset(dataset_X, label_X, save_dir_X, data_name)
del data_x, data_name, dataset_X, label_X

In [ ]:
video_dir = "/content/drive/My Drive/Videos/hall_monitor.y4m"
data_x, _, data_name = optflow_gen(video_dir)
dataset_X, label_X = dataset_generate(data_x)
save_dataset(dataset_X, label_X, save_dir_X, data_name)
del data_x, data_name, dataset_X, label_X